In [3]:
import os
import sys
import shutil
from pathlib import Path

sys.path.append('../src')

from utils import run_cmd

In [1]:
asm = '/media/GenomicResearch/Issue/UNSGM/denovo/Sample05/contigs.fasta'
raws = [
    '/media/GenomicResearch/Issue/UNSGM/raw_trim/Sample05_R1.fastq.gz',
    '/media/GenomicResearch/Issue/UNSGM/raw_trim/Sample05_R2.fastq.gz',
]
bam = '/media/GenomicResearch/Issue/UNSGM/denovo/Sample05/aln.sort.bam'
outdir = '/media/GenomicResearch/Issue/UNSGM/denovo/Sample05/pilon'

!bwa-mem2 index {asm}
!samtools faidx {asm}
!bwa-mem2 mem -v 3 -x intractg -t 16 {asm} {' '.join(raws)} | samtools sort --threads 16 -m 500m --reference {asm} -T /tmp/ -o {bam}
!samtools index {bam}
!conda run -n miaw pilon --genome {asm} --frags {bam} --minmq 60 --minqual 3 --fix bases --output pilon --outdir {outdir} --threads {threads} --changes --mindepth 0.25

Looking to launch executable "/usr/local/bin/bwa-mem2.avx2", simd = .avx2
Launching executable "/usr/local/bin/bwa-mem2.avx2"
[bwa_index] Pack FASTA... 1.04 sec
* Entering FMI_search
init ticks = 9168732749
ref seq len = 231674928
binary seq ticks = 4972071087
build suffix-array ticks = 195436162237
ref_seq_len = 231674928
count = 0, 66338276, 115837464, 165336652, 231674928
BWT[53189217] = 4
CP_SHIFT = 6, CP_MASK = 63
sizeof CP_OCC = 64
pos: 28959367, ref_seq_len__: 28959366
max_occ_ind = 3619920
build fm-index ticks = 29098128317
Total time taken: 120.9092
Looking to launch executable "/usr/local/bin/bwa-mem2.avx2", simd = .avx2
Launching executable "/usr/local/bin/bwa-mem2.avx2"
-----------------------------
Executing in AVX2 mode!!
-----------------------------
* SA compression enabled with xfactor: 8
* Ref file: /media/GenomicResearch/Issue/UNSGM/denovo/Sample05/contigs.fasta
* Entering FMI_search
* Index file found. Loading index from /media/GenomicResearch/Issue/UNSGM/denovo/Sam

In [4]:
def pilon_polish(fq_1, fq_2, reference, outdir, output='pilon', threads=4):
    bam = os.path.join(outdir, 'alignments.sort.bam')
    run_cmd(f"bwa index {reference}")
    run_cmd(f"samtools faidx  {reference}")
    
    cmd = f"bwa mem -v 3 -x intractg -t {threads} {reference} {fq_1} {fq_2} | "\
          f"samclip --ref {reference}.fai | "\
          f"samtools sort --threads {threads} -m 500m --reference {reference} -T /tmp/ -o {bam}"
    run_cmd(cmd)
    run_cmd(f"samtools index  {bam}")
    cmd = f"pilon --genome {reference} --frags {bam} --minmq 60 --minqual 3 --fix bases --output {output} "\
          f"--outdir {outdir} --threads {threads} --changes --mindepth 0.25"
    run_cmd(cmd)

In [6]:
fq_1 = '/media/NAS/MiSeqOutput/210305_M04855_0068_000000000-JDTYJ/Data/Intensities/BaseCalls/R14-0180-RifR_S42_L001_R1_001.fastq.gz'
fq_2 = '/media/NAS/MiSeqOutput/210305_M04855_0068_000000000-JDTYJ/Data/Intensities/BaseCalls/R14-0180-RifR_S42_L001_R2_001.fastq.gz'
reference = '/home/chen1i6c04/test/R14-0180-RifR.fa'

In [8]:
pilon_polish(fq_1, fq_2, reference, outdir='/home/chen1i6c04/test', threads=32)

In [9]:
p = run_cmd(f"wc -l /home/chen1i6c04/test/pilon.changes | awk '{{print $1}}'")

In [26]:
n = 0.4853185354

In [29]:
f'{n:.5f}'

'0.48532'